<a href="https://colab.research.google.com/github/NiravTech22/CaseCreatorAI/blob/main/Dummy_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np

# --- Configuration ---
GESTURES = ['flick_right', 'flick_left', 'up', 'down']
SAMPLES_PER_FILE = 100 # Number of IMU readings in one sample window
NUM_FILES_PER_GESTURE = 15 # How many unique CSV files to generate per gesture
DATA_DIR = 'data'

def generate_imu_data(gesture_name, num_samples):
    """
    Generates simulated IMU data (ax, ay, az, gx, gy, gz) based on a gesture.
    This simulates slightly different patterns for each gesture.
    """
    df = pd.DataFrame()

    # Base noise parameters
    noise_level = 0.1
    t = np.linspace(0, 1, num_samples) # Time index for pattern generation

    # Common IMU axes
    axes = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']

    # Generate data with small, distinct shifts based on gesture
    if gesture_name == 'flick_right':
        # Strong yaw (gy) and lateral acceleration (ay) spikes
        df['ax'] = np.sin(2 * np.pi * t * 2) * 0.2 + np.random.normal(0, noise_level, num_samples)
        df['ay'] = np.cos(np.pi * t) * 1.5 + np.random.normal(0, noise_level, num_samples) # Lateral acceleration
        df['az'] = np.random.normal(0, noise_level, num_samples)
        df['gx'] = np.random.normal(0, noise_level, num_samples)
        df['gy'] = np.sin(np.pi * t * 5) * 50 + np.random.normal(0, noise_level * 5, num_samples) # Yaw rotation
        df['gz'] = np.random.normal(0, noise_level, num_samples)

    elif gesture_name == 'flick_left':
        # Strong negative yaw (gy) and lateral acceleration (ay) spikes
        df['ax'] = np.sin(2 * np.pi * t * 2) * 0.2 + np.random.normal(0, noise_level, num_samples)
        df['ay'] = np.cos(np.pi * t) * -1.5 + np.random.normal(0, noise_level, num_samples) # Lateral acceleration (opposite)
        df['az'] = np.random.normal(0, noise_level, num_samples)
        df['gx'] = np.random.normal(0, noise_level, num_samples)
        df['gy'] = np.sin(np.pi * t * 5) * -50 + np.random.normal(0, noise_level * 5, num_samples) # Negative Yaw rotation
        df['gz'] = np.random.normal(0, noise_level, num_samples)

    elif gesture_name == 'up':
        # Strong upward acceleration (az) and pitch (gx)
        df['ax'] = np.random.normal(0, noise_level, num_samples)
        df['ay'] = np.random.normal(0, noise_level, num_samples)
        df['az'] = np.cos(np.pi * t) * 2.0 + np.random.normal(0, noise_level, num_samples) # Up/Down acceleration
        df['gx'] = np.sin(np.pi * t * 3) * 60 + np.random.normal(0, noise_level * 5, num_samples) # Pitch rotation
        df['gy'] = np.random.normal(0, noise_level, num_samples)
        df['gz'] = np.random.normal(0, noise_level, num_samples)

    elif gesture_name == 'down':
        # Strong downward (negative az) acceleration and pitch (gx)
        df['ax'] = np.random.normal(0, noise_level, num_samples)
        df['ay'] = np.random.normal(0, noise_level, num_samples)
        df['az'] = np.cos(np.pi * t) * -2.0 + np.random.normal(0, noise_level, num_samples) # Up/Down acceleration (opposite)
        df['gx'] = np.sin(np.pi * t * 3) * -60 + np.random.normal(0, noise_level * 5, num_samples) # Negative pitch rotation
        df['gy'] = np.random.normal(0, noise_level, num_samples)
        df['gz'] = np.random.normal(0, noise_level, num_samples)

    else:
        # Default/Rest pose: near zero with noise
        for axis in axes:
            df[axis] = np.random.normal(0, noise_level, num_samples)

    return df[axes]

def create_dummy_data():
    """Creates the data directory and populates it with simulated CSV files."""
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
        print(f"Created directory: {DATA_DIR}")

    total_files = 0
    for gesture in GESTURES:
        for i in range(1, NUM_FILES_PER_GESTURE + 1):
            file_name = f"{DATA_DIR}/{gesture}_{i:02d}.csv"
            data = generate_imu_data(gesture, SAMPLES_PER_FILE)
            data.to_csv(file_name, index=False)
            total_files += 1

    print(f"\nSuccessfully generated {total_files} dummy data files.")
    print("Your 'data/' directory is now ready for Step 3 and 4 (Feature Extraction and Training) of the ML pipeline.")

if __name__ == '__main__':
    create_dummy_data()


Successfully generated 60 dummy data files.
Your 'data/' directory is now ready for Step 3 and 4 (Feature Extraction and Training) of the ML pipeline.


In [ ]:
import os
import time
import pandas as pd
import numpy as np
# You will need to install 'pyserial' in your actual environment: pip install pyserial
# import serial # Commented out for simulation environment

# --- Configuration ---
DATA_DIR = 'data'
GESTURES = ['flick_right', 'flick_left', 'up', 'down']
SERIAL_PORT = 'COM3' # Change this to your Arduino's serial port (e.g., '/dev/ttyACM0' on Linux)
BAUD_RATE = 9600

# --- Fake Serial Class for Simulation (REPLACE THIS FOR REAL HARDWARE) ---
class FakeSerial:
    """Simulates a serial connection returning IMU data for a specific gesture."""
    def __init__(self, gesture_name, samples_per_file):
        print(f"--- Simulating Arduino for gesture: {gesture_name} ---")
        self.gesture_name = gesture_name
        self.samples_per_file = samples_per_file
        # Generate a batch of data to return, similar to the dummy data script
        self.data_generator = generate_imu_data(gesture_name, samples_per_file)
        self.current_sample_index = 0

    def readline(self):
        """Returns one line of simulated IMU data as a comma-separated string."""
        if self.current_sample_index >= self.samples_per_file:
            # Loop the data for continuous simulation
            self.current_sample_index = 0

        sample = self.data_generator.iloc[self.current_sample_index]
        self.current_sample_index += 1

        # Format: ax,ay,az,gx,gy,gz\n
        line = ",".join(f"{val:.4f}" for val in sample.values)

        # Simulate the time delay of a real serial port read
        time.sleep(0.01) # Simulates a 100Hz sampling rate

        return (line + '\n').encode('utf-8')

# Re-use the data generation function from the dummy script for the FakeSerial
def generate_imu_data(gesture_name, num_samples):
    """
    Generates simulated IMU data based on a gesture (simplified from the other script).
    Used to feed the FakeSerial object.
    """
    df = pd.DataFrame()
    noise_level = 0.05
    t = np.linspace(0, 1, num_samples)
    axes = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']

    # Example: Simple shift for 'flick_right'
    if gesture_name == 'flick_right':
        df['ax'] = np.random.normal(0, noise_level, num_samples)
        df['ay'] = 1.0 + np.random.normal(0, noise_level, num_samples)
        df['az'] = np.random.normal(0, noise_level, num_samples)
        df['gx'] = np.random.normal(0, noise_level, num_samples)
        df['gy'] = 40.0 * np.sin(2 * np.pi * t) + np.random.normal(0, 5, num_samples)
        df['gz'] = np.random.normal(0, noise_level, num_samples)

    # Add simple logic for other gestures if needed, or default to a baseline
    else:
        for axis in axes:
            df[axis] = np.random.normal(0, noise_level, num_samples)

    return df[axes]

# --- Main Collector Logic ---

def collect_live_data():
    """
    Connects to the serial port (or fake serial) and records IMU data
    for a specified gesture, saving it to a CSV file.
    """
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    print("--- Live Gesture Data Collector ---")
    print(f"Available Gestures: {GESTURES}")

    # 1. Get user input
    while True:
        gesture = input(f"Enter gesture to record (e.g., {GESTURES[0]}): ").lower().strip()
        if gesture in GESTURES:
            break
        print(f"Invalid gesture. Please choose one of: {GESTURES}")

    try:
        num_samples = int(input("Enter number of samples to record (e.g., 100): "))
    except ValueError:
        print("Invalid number. Defaulting to 100 samples.")
        num_samples = 100


    # 2. Initialize serial (use real serial when connected)
    try:
        # ser = serial.Serial(SERIAL_PORT, BAUD_RATE) # For real Arduino
        ser = FakeSerial(gesture, num_samples) # For simulation

        print(f"\nReady to record {num_samples} samples for '{gesture}'.")
        print("PERFORM THE GESTURE NOW!")

        # 3. Data collection loop
        data_window = []
        start_time = time.time()

        while len(data_window) < num_samples:
            try:
                line = ser.readline().decode('utf-8').strip()

                # Ensure the line is not empty and has the correct number of values
                if line and len(line.split(',')) == 6:
                    # Parse the 6 floating-point IMU values
                    ax, ay, az, gx, gy, gz = map(float, line.split(','))
                    data_window.append([ax, ay, az, gx, gy, gz])

                    if len(data_window) % 10 == 0:
                        print(f"Recorded {len(data_window)}/{num_samples} samples...", end='\r')

                # else:
                    # print(f"Skipping bad line: {line}") # Uncomment for debugging real serial

            except Exception as e:
                print(f"Error reading serial line: {e}")
                time.sleep(0.1) # Wait briefly on error

        end_time = time.time()

        # 4. Save data
        if data_window:
            df = pd.DataFrame(data_window, columns=['ax','ay','az','gx','gy','gz'])

            # Find the next available file index for this gesture
            i = 1
            file_path = f"{DATA_DIR}/{gesture}_{i:02d}.csv"
            while os.path.exists(file_path):
                i += 1
                file_path = f"{DATA_DIR}/{gesture}_{i:02d}.csv"

            df.to_csv(file_path, index=False)

            print(f"\n\n--- Recording Complete ---")
            print(f"Saved {len(data_window)} samples to: {file_path}")
            print(f"Time elapsed: {end_time - start_time:.2f} seconds.")

        else:
            print("\nRecording failed: No valid data collected.")

    except Exception as e:
        print(f"\nAn error occurred during serial initialization or communication: {e}")
        # print("Check if your Arduino is connected and the correct COM port is set.")


if __name__ == '__main__':
    collect_live_data()

--- Live Gesture Data Collector ---
Available Gestures: ['flick_right', 'flick_left', 'up', 'down']
Enter gesture to record (e.g., flick_right): down
Enter number of samples to record (e.g., 100): 500
--- Simulating Arduino for gesture: down ---

Ready to record 500 samples for 'down'.
PERFORM THE GESTURE NOW!
Recorded 500/500 samples...

--- Recording Complete ---
Saved 500 samples to: data/down_16.csv
Time elapsed: 5.38 seconds.


In [ ]:
import pandas as pd

file_path = 'data/flick_right_16.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Contents of {file_path}:")
    display(df)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Contents of data/flick_right_16.csv:


,ax,ay,az,gx,gy,gz
0,-0.0323,0.9219,-0.0265,-0.0342,8.2974,-0.0557
1,0.0477,1.0186,-0.0531,0.0613,4.2120,0.0277
2,-0.0145,0.9686,0.0400,0.0009,1.2269,-0.0140
3,0.1116,1.0359,-0.0297,0.0338,9.5926,-0.0394
4,-0.0128,1.0155,0.0144,0.0558,21.4574,-0.0356
...,...,...,...,...,...,...
95,-0.0230,1.0416,0.0266,0.0741,-9.5097,0.0167
96,-0.0460,1.0557,-0.0204,0.0203,-3.0270,-0.0696
97,-0.0590,0.9563,-0.0131,0.1139,-1.1791,-0.0032
98,-0.0438,1.0256,-0.0130,0.0257,-0.4570,-0.0097
